In [1]:
# import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import quantfreedom as qf
import numpy as np
import pandas as pd
import talib
from itertools import product
from talib import abstract
from quantfreedom.backtester.enums.enums import *
from quantfreedom.backtester.base.base import *

np.set_printoptions(precision=4)
pd.options.plotting.backend = 'plotly'

In [2]:
# data = qf.CCXTData.data_download(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-02-23T00:00:00Z',
#     symbol='BTCUSDT',
#     timeframe='30m',
# )
# data.to_csv('30min.csv')
data = pd.read_csv('30min.csv', index_col='time')

In [3]:
entries = rsi_below_entries(
    timeperiods=np.arange(15, 31, 5).tolist(),
    below_ranges=np.arange(20, 31, 5).tolist(),
    prices=data.close,
)

In [4]:
account_state = AccountAndTradeState(
    available_balance=1000.,
    equity=1000.,
)
order = OrderEverything(
    lev_mode=np.array([LeverageMode.LeastFreeCashUsed]),
    order_type=np.array([OrderType.LongEntry]),
    size_type=np.array([SizeType.RiskPercentOfAccount]),
    size_pct=np.array([1.]),
    max_equity_risk_pct=np.arange(3, 6.1, 1),
    sl_pcts=np.arange(3, 6.1, .25),
    risk_rewards = np.arange(3, 6.1, .5),
)

In [5]:
df_array, order_records, _ = cart_from_signals(
    open=data.open,
    high=data.high,
    low=data.low,
    close=data.close,
    entries=entries,
    order=order,
    account_state=account_state,
)

In [6]:
df = pd.DataFrame(df_array)
pd.options.display.float_format = '{:,.2f}'.format

for idx, name in enumerate(entries.columns.names):
    df[name] = df['ind_set']
    for jjj in range(df['ind_set'].min(), df['ind_set'].max()+1):
        df[name].mask(df['ind_set'] == jjj, entries.columns[jjj][idx], inplace=True)
df.sort_values(by=['to_the_upside'], ascending=False).head(30)

,or_set,ind_set,total_trades,gain_pct,win_rate,to_the_upside,total_fees,total_pnl,ending_eq,sl_pct,rr,max_eq_risk,rsi_below,rsi_timeperiod
22,327,2,34.00,53.30,23.53,0.26,45.99,533.01,"1,533.01",3.50,5.00,6.00,20,25
21,326,2,36.00,51.83,22.22,0.21,49.68,518.32,"1,518.32",3.25,5.00,6.00,20,25
12,236,2,34.00,51.03,23.53,0.16,40.67,510.27,"1,510.27",3.50,5.00,5.00,20,25
23,328,2,31.00,154.86,29.03,0.11,56.74,"1,548.65","2,548.65",3.75,5.00,6.00,20,25
24,339,2,36.00,68.74,22.22,0.11,52.77,687.43,"1,687.43",3.25,5.50,6.00,20,25
4,146,2,31.00,113.87,29.03,0.10,39.59,"1,138.71","2,138.71",3.75,5.00,4.00,20,25
25,340,2,32.00,68.22,21.88,0.10,45.42,682.23,"1,682.23",3.50,5.50,6.00,20,25
13,237,2,31.00,134.01,29.03,0.09,49.75,"1,340.09","2,340.09",3.75,5.00,5.00,20,25
9,198,2,31.00,55.70,29.03,0.06,39.18,557.04,"1,557.04",3.75,3.50,5.00,20,25
26,352,2,35.00,74.76,20.00,0.05,52.83,747.64,"1,747.64",3.25,6.00,6.00,20,25
